In [18]:
import requests
from bs4 import BeautifulSoup

url = 'http://books.toscrape.com/'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')


In [19]:
books = soup.find_all('article', class_='product_pod')

book_list = []

for book in books:
    # Title
    title = book.h3.a['title']
    
    # Price
    price = book.find('p', class_='price_color').text
    
    # Availability
    availability = book.find('p', class_='instock availability').text.strip()
    
    # Data Dictionary
    book_data = {
        'Title': title,
        'Price': price,
        'Availability': availability
    }
    
    book_list.append(book_data)


In [20]:
import time

book_list = []

for page in range(1, 51):  # Assuming there are 50 pages
    url = f'http://books.toscrape.com/catalogue/page-{page}.html'
    response = requests.get(url)
    
    if response.status_code != 200:
        break  # Exit the loop if we've reached the last page
    
    soup = BeautifulSoup(response.content, 'html.parser')
    books = soup.find_all('article', class_='product_pod')
    
    for book in books:
        title = book.h3.a['title']
        price = book.find('p', class_='price_color').text
        availability = book.find('p', class_='instock availability').text.strip()
        
        book_data = {
            'Title': title,
            'Price': price,
            'Availability': availability
        }
        
        book_list.append(book_data)
    
    time.sleep(1)  # Be polite and don't spam requests too rapidly


In [22]:
import pandas as pd

df = pd.DataFrame(book_list)
df.to_csv('books_data.csv', index=False)
